In [26]:
import random
from textblob.classifiers import NaiveBayesClassifier
from nltk import tokenize
import pandas as pd
import numpy as np
from textblob import TextBlob

In [27]:
df_train = pd.read_json('../input/ted_ru-20160408.json.train')

In [28]:
df_train.head()

,@id,class,content,head
0,1848,2,"Я был неописуемо удивлён, узнав о существовани...","{'videopath': 'talks/NeilTurok_2008.mp4', 'tal..."
1,881,2,Я работаю с бактериями. Сейчас я покажу вам сд...,"{'videopath': 'talks/DamianPalin_2012U.mp4', '..."
2,1757,1,"Привет, я здесь чтобы поговорить с вами о важн...","{'videopath': 'talks/LauraTrice_2008.mp4', 'ta..."
3,1199,1,Я глава отдела развития общественной организац...,"{'videopath': 'talks/MarkBezos_2011U.mp4', 'ta..."
4,1855,6,"Я думаю, мой взгляд на простоту можно отлично ...","{'videopath': 'talks/AlanKay_2007.mp4', 'talki..."


In [29]:
df_train.head()['head'][1]

{'charnum': 2214,
 'content-type': 'text/html; charset=utf-8',
 'date': '2012/02/27',
 'description': 'TED Talk Subtitles and Transcript: Человечеству необходимо всё больше и больше чистой воды; мы берём её из океана, опресняем и пьём. Но как быть с оставшимся солевым раствором? В этом увлекательном коротком докладе стипендиат TED Дэмиан Палин предлагает решение: добывать из него необходимые нам минералы с помощью металлопотребляющих бактерий.',
 'dtime': 'Thu Jan 28 14:25:01 CET 2016',
 'encoding': 'UTF-8',
 'keywords': ['talks',
  'TED Conference',
  'TED Fellows',
  'mining',
  'science',
  'technology',
  'water'],
 'pagesize': 84326,
 'speaker': 'Damian Palin',
 'talkid': 1470,
 'title': 'Дэмиан Палин: Добыча минералов из морской воды',
 'url': 'http://www.ted.com/talks/damian_palin_mining_minerals_from_seawater',
 'videopath': 'talks/DamianPalin_2012U.mp4',
 'videourl': 'http://download.ted.com/talks/DamianPalin_2012U.mp4',
 'wordnum': 306}

In [30]:
train = [row['head']['description'][35:] for index, row in df_train.iterrows()]

In [31]:
df_train['description'] = pd.Series(train)

In [47]:
train_set = [
    (sentence, label) for label in np.unique(df_train['class']) for sentence in 
    tokenize.sent_tokenize(np.add.reduce(df_train[df_train['class'] == label]['description'].values))
]
random.shuffle(train_set)

In [ ]:
nb_clf = NaiveBayesClassifier(train_set)

In [16]:
test_df = pd.read_json('../input/ted_ru-20160408.json.test')

In [41]:
test_set = [row['head']['description'][35:] for index, row in test_df.iterrows()]

In [42]:
test_set[0]

'Политический эксперт Сэлли Кон говорит, что настало время, когда либералы и консерваторы должны переступить через барьер своих различий и действительно прислушаться друг к другу. В своей оптимистической речи Кон делится тем, что она узнала, будучи прогрессивной лесбиянкой, а так же ведущей на Fox News. Она заявляет, что значение имеет не политическая, а эмоциональная корректность.'

In [ ]:
predictions = []

for description in test_set:
    text_blob = TextBlob(description, classifier=nb_clf)
    predictions.append(text_blob.classify())

In [44]:
sample_submission = pd.read_csv('../input/ted_ru-20160408.json.submission2')
sample_submission['class'] = pd.Series(predictions)
sample_submission.to_csv('naive_bayes_on_description_submission.csv', index=False)